# Extraction of SWR from CA1 recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [10]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
%matplotlib widget

folder_base = Path('/Users/ahay/Documents/DATA/OPNE/ArchTshort/OD2/')
filename = folder_base / f'LFPwake0.npy'
filename3 = folder_base / f'LFPwakeremoved.npy'
filename2 = folder_base / f'RawDataChannelExtractedDS.npy'
EMGbooleaninput = folder_base / f'EMGframeBoolean.pkl'

EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremoved = np.load(filename3, mmap_mode= 'r')
All = np.load(filename2, mmap_mode= 'r')
All = All.T
CA1 = All[0:10000000, 0]-All[0:10000000, 1]
CA1wakeremoved = LFPwakeremoved[0:10000000, 0]-LFPwakeremoved[0:10000000,1]
EMGboolean = EMGboolean[0:10000000]
CA1.shape

(942610,)

# Band pass filter
        SWR: 120-200 Hz

In [11]:
# Filtre parameter:
f_lowcut = 120.
f_hicut = 200.
fs = 1000
nyq = 0.5 * fs
N = 6                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_CA1 = signal.filtfilt(b, a, CA1)
filt_CA1wakeremoved = signal.filtfilt(b, a, CA1wakeremoved)

# Plot
#times = np.arange(0, CA1.size/fs, 1./fs)
#timesmin = np.arange(0, CA1.size/fs/60, 1./fs/60)
#fig, ax = plt.subplots()
#ax.plot(timesmin, filt_CA1)

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [12]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(120, 200, 80)
widths = w*fs / (2*freq*np.pi)
CA1NWcwt = signal.cwt(CA1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation
absCA1NWcwt = np.absolute(CA1NWcwt)
proj_CA1NWcwt = np.sum(absCA1NWcwt, axis = 0)/80
sdproj_CA1cwt = np.std(proj_CA1NWcwt)
msdproj_CA1cwt = np.mean(proj_CA1NWcwt)
sd3proj_CA1cwt = msdproj_CA1cwt + sdproj_CA1cwt*3
sd05proj_CA1cwt = msdproj_CA1cwt + sdproj_CA1cwt*0.5
sd1proj_CA1cwt = msdproj_CA1cwt + sdproj_CA1cwt

In [6]:
from scipy import stats

# freq = np.arange(120, 200, 1)
# CA1NWcwtN = (CA1NWcwt.T*freq).T

zCA1NWcwtN = stats.zscore(CA1NWcwtN, axis=None)
sdtest = np.std(zCA1NWcwtN, axis=0)
# absCA1NWcwt = np.absolute(CA1NWcwt)
# proj_CA1NWcwt = np.sum(absCA1NWcwt, axis = 0)/80
# sdproj_CA1cwt = np.std(proj_CA1NWcwt)
sdmax = np.multiply(sdCA1,proj_CA1cwtt)/50

# Defining subset
start = 00000
end = 80000

times = np.arange(0, CA1.size/fs, 1./fs)
tt = times[start:end]
CA1t = CA1wakeremoved[start:end]/2+3000
sdmaxt = sdmax[start:end]
proj_CA1cwtt = proj_CA1NWcwt[start:end]
plt.close('all')
plt.plot(tt, CA1t)
plt.plot(tt, sdmaxt)
plt.show()

msdproj_CA1cwt = np.mean(sdmax)
sd3proj_CA1cwt = msdproj_CA1cwt + sdmax*3
sd05proj_CA1cwt = msdproj_CA1cwt + sdmax*0.5
sd1proj_CA1cwt = msdproj_CA1cwt + sdmax

NameError: name 'CA1NWcwtN' is not defined

In [6]:

plt.imshow(zCA1NWcwtN, extent=[0, 0, 100, 100], cmap='PRGn', aspect='auto',
           vmax=abs(zCA1NWcwtN).max(), vmin=-abs(zCA1NWcwtN).max())

plt.show()

NameError: name 'zCA1NWcwtN' is not defined

Second on the signal for which wake times have been zeroed

In [13]:
# Conservative boolean filtering of CA1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fCA1wake0C = filt_CA1.copy()
fCA1wake0C[BooleanCons] = 0
CA1wake0C = CA1.copy()
CA1wake0C[BooleanCons] = 0
# Liberal boolean filtering of CA1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fCA1wake0L = filt_CA1.copy()
fCA1wake0L[BooleanLib] = 0
CA1wake0L = CA1.copy()
CA1wake0L[BooleanLib] = 0

# Computation of CWT
CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absCA1W0Ccwt = np.absolute(CA1cwtWake0cons)
proj_CA1W0Ccwt = np.sum(absCA1W0Ccwt, axis = 0)/80
absCA1W0Lcwt = np.absolute(CA1cwtWake0lib)
proj_CA1W0Lcwt = np.sum(absCA1W0Lcwt, axis = 0)/80

## Extracting SWRs and determining main properties 

First extraction of SWR peaks, initiation, end and width

In [14]:
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

# 3 sd threshold
peaks, properties = find_peaks(proj_CA1W0Lcwt, prominence=1, width=20, height=sd3proj_CA1cwt)
properties["prominences"], properties["widths"]

# SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CA1W0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

Second extraction of main frequency and power 

In [15]:
projMaxP_cwtmg = np.max(CA1cwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(CA1cwtWake0lib, axis = 0) + 120
projMaxP_cwtmg.shape

nb_SWR = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_SWR:
    SWR_start = int(SWR_prop[3,tt])
    SWR_stop = int(SWR_prop[4,tt])
    SWR_MaxP = projMaxP_cwtmg[SWR_start:SWR_stop]
    SWR_MaxF = projMaxF_cwtmg[SWR_start:SWR_stop]
    data[tt, 0] = max(SWR_MaxF).round()
    data[tt, 1] = max(SWR_MaxP).round()
    data[tt, 2] = round(sum(SWR_MaxF)/len(SWR_MaxF))
    data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))

param_SWR = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSWR_prop = SWR_prop.transpose()
pd_prop_SWR = pd.DataFrame(tSWR_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_SWR = pd.concat([pd_prop_SWR, param_SWR], axis=1)

/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_24366/3356834751.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(SWR_MaxP).round()
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_24366/3356834751.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_24366/3356834751.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))


### Store the results in All_SWR_prop pd dataframe and save as pkl/csv for post processing.

End of Notebook. 

In [16]:
filename2 = folder_base / f'SWRproperties.pkl'
filename3 = folder_base / f'SWRproperties.csv'
All_SWR.to_pickle(filename2)
All_SWR.to_csv(filename3, sep = ',')


combined = np.stack([fCA1wake0C, proj_CA1W0Ccwt], axis = 1)
filenameC = folder_base / f'SignalCA1.npy'
np.save(filenameC, combined)

# if done and no intention to display for assessment
#%reset
#plt.close('all')

### Display

ephys viewer to check SWR detection

In [17]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
TTL = All[0:10000000, 11]#
combined = np.stack([CA1, fCA1wake0C, proj_CA1W0Lcwt, proj_CA1W0Ccwt, TTL], axis = 1)

app = mkQApp()

sample_rate = 1000.
t_start = 0.

SWR_peak = peaks
SWR_start = SWR_prop[3,:].astype(int)
SWR_end = SWR_prop[4,:].astype(int)

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: SWR_peak, 1: SWR_start, 2: SWR_end}
#and asign them to some channels each
scatter_channels = {0: [0, 2], 1: [0, 1], 2: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

view1.by_channel_params['ch0', 'gain'] = 0.001
view1.by_channel_params['ch1', 'gain'] = 0.002
view1.by_channel_params['ch2', 'gain'] = 0.002
view1.by_channel_params['ch3', 'gain'] = 0.005
view1.by_channel_params['ch4', 'gain'] = 0.001

view1.by_channel_params['ch0', 'offset'] = 1
view1.by_channel_params['ch1', 'offset'] = -0
view1.by_channel_params['ch2', 'offset'] = -1
view1.by_channel_params['ch3', 'offset'] = -1
view1.by_channel_params['ch4', 'offset'] = 2
#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  5.1021575927734375e-05 s
refresh duration for  0.0005428791046142578 s
refresh duration for  0.0002727508544921875 s
refresh duration for  0.00028514862060546875 s
refresh duration for  0.00026798248291015625 s
refresh duration for  0.0002627372741699219 s
refresh duration for  0.00026106834411621094 s
refresh duration for  0.00021982192993164062 s
refresh duration for  0.00026702880859375 s
refresh duration for  0.00027179718017578125 s
refresh duration for  0.0002498626708984375 s
refresh duration for  0.00023317337036132812 s
refresh duration for  0.00023603439331054688 s
refresh duration for  0.00022983551025390625 s
refresh duration for  0.0002391338348388672 s
refresh duration for  0.000209808349609375 s
refresh duration for  0.0002601146697998047 s
refresh duration for  0.0002300739288330078 s
refresh duration for  0.00021314620971679688 s
refresh duration for  0.0002129077911376953 s
refresh duration for  0.0002651214599609375 s
re

0

: 

Quick plot of average and max intensity (can be done with freq as well) paired.

In [ ]:

# input data:
MaxSWR = param_SWR['Max int']
AvgSWR = param_SWR['Avg int']

# plotting the points
plt.scatter(np.zeros(len(MaxSWR)), MaxSWR)
plt.scatter(np.ones(len(AvgSWR)), AvgSWR)

# plotting the lines
for i in range(len(MaxSWR)):
    plt.plot( [0,1], [MaxSWR[i], AvgSWR[i]], c='k')

plt.xticks([0,1], ['Max', 'Avg'])
plt.show()